# Preprocessing Twitter Data

In [1]:
# import libraries

import numpy as np
import pandas as pd  
import csv, re
import nltk
from langdetect import detect
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


In [2]:
# setting some options of pandas
    
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 10)

%pastebin 1-2

In [3]:
df = pd.read_csv("../Data_json_csv/Twitter/D_12_Oct_24_Oct.csv", encoding = "latin1")

In [11]:
df

,user,fullname,tweet-id,timestamp,url,likes,replies,retweets,text,html
0,CristiCiobo,Crristic,1052710849843011585,2018-10-17 23:59:59,/CristiCiobo/status/1052710849843011585,0,0,0,Just buy an iphone? Bonus points for enhanced security (never trusted android),"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Just buy an iphone? Bonus points for enhanced security (never trusted android)</p>"
1,Roy2316Team,Roy project,1052710849649860608,2018-10-17 23:59:59,/Roy2316Team/status/1052710849649860608,0,0,0,easy vote. voted,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">easy vote. voted</p>"
2,chicken_yaroo_,ãã­ã³ééÂ ð@BTCFXãç¥ããã¬ã¼ãã¼,1052710840502108163,2018-10-17 23:59:57,/chicken_yaroo_/status/1052710840502108163,1,1,0,BTC FX\r\n\r\nCryptowatchãbfFXã ãä½ããããããã pic.twitter.com/huMJj0C92n,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""ja""><strong>BTC</strong> FX\r\n\r\nCryptowatchãbfFXã ãä½ããããããã <a class=""twitter-timeline-link u-hidden"" data-pre-embedded=""true"" dir=""ltr"" href=""https://t.co/huMJj0C92n"">pic.twitter.com/huMJj0C92n</a></p>"
3,ohiobitcoin,Ohio Bitcoin Â ðÂ ð°Â ð,1052710820956839936,2018-10-17 23:59:52,/ohiobitcoin/status/1052710820956839936,1,0,0,Rwanda Starts Tracking Conflict Metal Tantalum With Blockchain https://goo.gl/dLPmUEÂ #bitcoin #altcoin #bitcoinprice #ethereum #airdrop,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Rwanda Starts Tracking Conflict Metal Tantalum With Blockchain <a class=""twitter-timeline-link"" data-expanded-url=""https://goo.gl/dLPmUE"" dir=""ltr"" href=""https://t.co/QwPvv9e6NP"" rel=""nofollow noopener"" target=""_blank"" title=""https://goo.gl/dLPmUE""><span class=""tco-ellipsis""></span><span class=""invisible"">https://</span><span class=""js-display-url"">goo.gl/dLPmUE</span><span class=""invisible""></span><span cla..."
4,BitcoinBlockNtf,BitcoinBlockNotifier,1052710811276328960,2018-10-17 23:59:50,/BitcoinBlockNtf/status/1052710811276328960,0,0,0,-=[ 546.211 ]=-\r\n\r\nTxs: 1.091\r\nSize: 652.53 KB\r\nStripped: 494.29 KB\r\nTime: 1539820785\r\n\r\nReward: 12.5 BTC\r\nFees: 0.06051360 BTC\r\nMiner: SlushPool\r\n\r\nMempool: 454 txs,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">-=[ 546.211 ]=-\r\n\r\nTxs: 1.091\r\nSize: 652.53 KB\r\nStripped: 494.29 KB\r\nTime: 1539820785\r\n\r\nReward: 12.5 <strong>BTC</strong>\r\nFees: 0.06051360 <strong>BTC</strong>\r\nMiner: SlushPool\r\n\r\nMempool: 454 txs</p>"
...,...,...,...,...,...,...,...,...,...,...
411507,LaMarkazia,La Markazia | ÙØ§Ù Ø±ÙØ²ÙØ©,1051623688804528128,2018-10-15 00:00:00,/LaMarkazia/status/1051623688804528128,0,0,0,"Ø£Ø³Ø¹Ø§Ø± Ø§ÙØ¹Ù ÙØ§Øª Ø§ÙØ±ÙÙ ÙØ©\r\n\r\nÙ­ $BTC Ø¨ØªÙÙÙÙ 6,291.93 Ø¯ÙÙØ§Ø± -0.33%\r\nÙ­ $ETH Ø§ÙØ«ÙØ±ÙÙÙ 195.66 Ø¯ÙÙØ§Ø± -1.19%\r\nÙ­ $XRP Ø±ÙØ¨Ù 0.401757811 Ø¯ÙÙØ§Ø± -3.81% \r\nÙ­ $BCH Ø¨ØªÙÙÙÙ ÙØ§Ø´ 439.25 Ø¯ÙÙØ§Ø± -1.04%\r\nÙ­ $EOS EOS 5.17 Ø¯ÙÙØ§Ø± -0.50%","<p class=""TweetTextSize js-tweet-text tweet-text tweet-text-rtl"" data-aria-label-part=""0"" lang=""ar"">Ø£Ø³Ø¹Ø§Ø± Ø§ÙØ¹Ù ÙØ§Øª Ø§ÙØ±ÙÙ ÙØ©\r\n\r\nÙ­ <img alt=""ð»"" aria-label=""Emoji: Down-pointing red triangle"" class=""Emoji Emoji--forText"" draggable=""false"" src=""https://abs.twimg.com/emoji/v2/72x72/1f53b.png"" title=""Down-pointing red triangle""/> <a class=""twitter-cashtag pretty-link js-nav"" data-query-source=""cashtag_click"" dir=""ltr"" href=""/search?q=%24BTC&amp;src=ctag""><s>$</s><b><strong..."
411508,CryptoGulp,Crypto Gulp,1051623688750211073,2018-10-15 00:00:00,/CryptoGulp/status/1051623688750211073,0,0,0,"Top 5 Cryptocurrencies - Current Prices\r\n\r\nBitcoin $BTC: $6,290.93 -0.34%\r\nEthereum $ETH: $195.71 -1.16% \r\nXRP $XRP: $0.402769919 -3.51% \r\nBitcoin Cash $BCH: $439.23 -1.05% \r\nEOS $EOS: $

### Understanding data

##to drop first row(0) with blank values
df.drop(0,axis=0,inplace=True)


### Analyze dimensions

In [5]:
columns = df.columns
size = df.size
shape = df.shape
# default print
info = df.info()

print("\n"+"size(rows*columns) - "+str(size)+"\n")
print("columns - "+str(columns)+"\n")
print("shape - "+str(shape)+"\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411512 entries, 0 to 411511
Data columns (total 10 columns):
user         411512 non-null object
fullname     411491 non-null object
tweet-id     411512 non-null int64
timestamp    411512 non-null object
url          411512 non-null object
likes        411512 non-null int64
replies      411512 non-null int64
retweets     411512 non-null int64
text         411224 non-null object
html         411512 non-null object
dtypes: int64(4), object(6)
memory usage: 31.4+ MB

size(rows*columns) - 4115120

columns - Index(['user', 'fullname', 'tweet-id', 'timestamp', 'url', 'likes', 'replies',
       'retweets', 'text', 'html'],
      dtype='object')

shape - (411512, 10)



### Handel Missing values 

In [6]:
df.isnull().values.any()

True

In [9]:
# df.dropna(how='any', inplace = 'true')
# df.dropna(subset = ['text'], how = 'any', inplace = True)

df.isnull().sum()

user          0
fullname     21
tweet-id      0
timestamp     0
url           0
likes         0
replies       0
retweets      0
text          0
html          0
dtype: int64

In [8]:
# Drop rows with missing values in text columns
df['text'].replace("", np.nan, inplace=True)
df.dropna(subset=['text'], inplace=True)


### Handel Duplicate Values

In [12]:
print(df.duplicated().all())
# df.drop_duplicates(keep="first", inplace = true).shape

False


In [16]:
df
df = df[df.likes > 0]
df = df[df.replies > 0]
df = df[df.retweets > 0]
df.head()
df.reset_index(inplace = True, drop = True)
df

,user,fullname,tweet-id,timestamp,url,likes,replies,retweets,text,html
0,SpritzCoin,SpritzCoin,1052710680200134657,2018-10-17 23:59:19,/SpritzCoin/status/1052710680200134657,13,12,12,Good morning #Tokyo\r\n#Week42 weekly #airdrop!\r\nLAST SO BIG #SpritzCoin $sprtz\r\n1. RT this one\r\n2. Fill in\r\nhttps://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦\r\n#crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum\r\n$BTC $ETH\r\n#NewAgepic.twitter.com/ujOUhD8vIy,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Good morning <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Tokyo?src=hash""><s>#</s><b>Tokyo</b></a>\r\n<a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Week42?src=hash""><s>#</s><b>Week42</b></a> weekly <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href..."
1,crashfoxcrypto,CrashFoxâ¢,1052710224199671808,2018-10-17 23:57:30,/crashfoxcrypto/status/1052710224199671808,10,2,1,"Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy bitcoin & silver. Vape some kush... It's good karma. Best wishes & regards from ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""4"" lang=""en"">Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy <strong>bitcoin</strong> &amp; silver. Vape some kush... It's good karma. Best wishes &amp; regards from ya boy Crash. <img alt=""â"" aria-label=""Emoji: Victory hand"" class=""Emoji Emoji--forText"" draggable=""false"" src=""https://abs.twimg.com/emoji/v2/72x72/270c.png"" title=""Victory hand""/> <a class=""twitter-tim..."
2,AllDef,All Def,1052710193841102849,2018-10-17 23:57:23,/AllDef/status/1052710193841102849,30,12,1,"""Do you take Bitcoin?"" pic.twitter.com/ITv8UdkCZv","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">""Do you take <strong>Bitcoin</strong>?"" <a class=""twitter-timeline-link u-hidden"" data-pre-embedded=""true"" dir=""ltr"" href=""https://t.co/ITv8UdkCZv"">pic.twitter.com/ITv8UdkCZv</a></p>"
3,BitcoinCensus,Dr. Josh Cotton,1052709336357789696,2018-10-17 23:53:58,/BitcoinCensus/status/1052709336357789696,27,2,3,Great to meet you today @MarkYusko during the Digital Asset Strategies Summit! I loved your presentation and your #bullish outlook on #Bitcoin! pic.twitter.com/XcO5Y8fpH4,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Great to meet you today <a class=""twitter-atreply pretty-link js-nav"" data-mentioned-user-id=""254280603"" dir=""ltr"" href=""/MarkYusko""><s>@</s><b>MarkYusko</b></a> during the Digital Asset Strategies Summit! I loved your presentation and your <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/bullish?src=hash""><s>#</s><b>bullish</b></a> outlook on <a class=..."
4,thecryptokidd,CryptoKid,1052708492899946496,2018-10-17 23:50:37,/thecryptokidd/status/1052708492899946496,68,5,10,Bitcoin Cash was the best thing to happen to Bitcoin since the Bitcoin white paper.,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en""><strong>Bitcoin</strong> Cash was the best thing to happen to <strong>Bitcoin</strong> since the <strong>Bitcoin</strong> white paper.</p>"
...,...,...,...,...,...,...,...,...,...,...
15236,thesignalio,The Signal,1051623975401480192,2018-10-15 00:01:08,/thesignalio/status/1051623975401480192,2,1,1,Signal: #QTUM (#QTUM) $QTUM\r\n\r\nBuy Range : < 0.00055000 (buy below)\r\n\r\nTarget 1 : 61000 ~11% \r\nTarget 2 : 75000 ~3

### Other Languages

In [18]:

count = 0
# for feature not found
count2 = 0 
for i in range(1, len(df), 2):
#     print(i)
    try:
        if (detect(str(df.loc[i,'text'])) != "en"):
            df.drop(i, axis = 0, inplace = True)
            count +=1

    except Exception:
        count2 +=1
print("other language "+str(count))
print("feature not found for "+str(count2))


other language 1028
feature not found for 0


In [19]:
df.shape

(14213, 10)

df.drop(81230,axis=0,inplace=True)
print(detect("Catherine Flick  Bitcoin Dual Use Dilemma   Bitcoin Media   bitcoin"))

### Handle Stopwords

In [20]:
# download this once
# import nltk
# nltk.download('stopwords')

stop = stopwords.words("english")
print(df['text']+"\n")
df['step1'] =  df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df['step1'])


0        Good morning #Tokyo\r\n#Week42 weekly #airdrop!\r\nLAST SO BIG #SpritzCoin $sprtz\r\n1. RT this one\r\n2. Fill in\r\nhttps://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦\r\n#crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum\r\n$BTC $ETH\r\n#NewAgepic.twitter.com/ujOUhD8vIy\n
1                                                                                                                                                                   Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy bitcoin & silver. Vape some kush... It's good karma. Best wishes & regards from ya boy Crash.  https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦\n
2                                                                                                                                                   

### Handle Abbreviations

In [21]:
df

,user,fullname,tweet-id,timestamp,url,likes,replies,retweets,text,html,step1
0,SpritzCoin,SpritzCoin,1052710680200134657,2018-10-17 23:59:19,/SpritzCoin/status/1052710680200134657,13,12,12,Good morning #Tokyo\r\n#Week42 weekly #airdrop!\r\nLAST SO BIG #SpritzCoin $sprtz\r\n1. RT this one\r\n2. Fill in\r\nhttps://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦\r\n#crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum\r\n$BTC $ETH\r\n#NewAgepic.twitter.com/ujOUhD8vIy,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Good morning <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Tokyo?src=hash""><s>#</s><b>Tokyo</b></a>\r\n<a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Week42?src=hash""><s>#</s><b>Week42</b></a> weekly <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href...",Good morning #Tokyo #Week42 weekly #airdrop! LAST SO BIG #SpritzCoin $sprtz 1. RT one 2. Fill https://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦ #crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum $BTC $ETH #NewAgepic.twitter.com/ujOUhD8vIy
1,crashfoxcrypto,CrashFoxâ¢,1052710224199671808,2018-10-17 23:57:30,/crashfoxcrypto/status/1052710224199671808,10,2,1,"Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy bitcoin & silver. Vape some kush... It's good karma. Best wishes & regards from ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""4"" lang=""en"">Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy <strong>bitcoin</strong> &amp; silver. Vape some kush... It's good karma. Best wishes &amp; regards from ya boy Crash. <img alt=""â"" aria-label=""Emoji: Victory hand"" class=""Emoji Emoji--forText"" draggable=""false"" src=""https://abs.twimg.com/emoji/v2/72x72/270c.png"" title=""Victory hand""/> <a class=""twitter-tim...","Shit like makes smile. Positivity spreads...do good, good people, stop fucking up, buy bitcoin & silver. Vape kush... It's good karma. Best wishes & regards ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦"
2,AllDef,All Def,1052710193841102849,2018-10-17 23:57:23,/AllDef/status/1052710193841102849,30,12,1,"""Do you take Bitcoin?"" pic.twitter.com/ITv8UdkCZv","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">""Do you take <strong>Bitcoin</strong>?"" <a class=""twitter-timeline-link u-hidden"" data-pre-embedded=""true"" dir=""ltr"" href=""https://t.co/ITv8UdkCZv"">pic.twitter.com/ITv8UdkCZv</a></p>","""Do take Bitcoin?"" pic.twitter.com/ITv8UdkCZv"
3,BitcoinCensus,Dr. Josh Cotton,1052709336357789696,2018-10-17 23:53:58,/BitcoinCensus/status/1052709336357789696,27,2,3,Great to meet you today @MarkYusko during the Digital Asset Strategies Summit! I loved your presentation and your #bullish outlook on #Bitcoin! pic.twitter.com/XcO5Y8fpH4,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Great to meet you today <a class=""twitter-atreply pretty-link js-nav"" data-mentioned-user-id=""254280603"" dir=""ltr"" href=""/MarkYusko""><s>@</s><b>MarkYusko</b></a> during the Digital Asset Strategies Summit! I loved your presentation and your <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/bullish?src=hash""><s>#</s><b>bullish</b></a> outlook on <a class=...",Great meet today @MarkYusko Digital Ass

In [22]:
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "D:\\BE_Project\\notebooks\\slangs.txt"
        # File Access mode [Read Mode]
        accessMode = "r"
        with open(fileName, accessMode) as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    # Replacing commas with spaces for final output.
    return ' '.join(user_string)
#     return user_string


    

In [23]:
df['step1'] = df['step1'].apply(lambda x: translator(x))
print(df['step1'])

0        Good morning #Tokyo #Week42 weekly #airdrop! LAST SO BIG #SpritzCoin $sprtz 1. RT one 2. Fill https://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦ #crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum $BTC $ETH #NewAgepic.twitter.com/ujOUhD8vIy
1                                                                                                                                                            Shit like makes smile. Positivity spreads...do good, good people, stop fucking up, buy bitcoin & silver. Vape kush... It's good karma. Best wishes & regards ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦
2                                                                                                                                                                                                                       

In [24]:
# need a map for this i guess
df

,user,fullname,tweet-id,timestamp,url,likes,replies,retweets,text,html,step1
0,SpritzCoin,SpritzCoin,1052710680200134657,2018-10-17 23:59:19,/SpritzCoin/status/1052710680200134657,13,12,12,Good morning #Tokyo\r\n#Week42 weekly #airdrop!\r\nLAST SO BIG #SpritzCoin $sprtz\r\n1. RT this one\r\n2. Fill in\r\nhttps://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦\r\n#crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum\r\n$BTC $ETH\r\n#NewAgepic.twitter.com/ujOUhD8vIy,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Good morning <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Tokyo?src=hash""><s>#</s><b>Tokyo</b></a>\r\n<a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Week42?src=hash""><s>#</s><b>Week42</b></a> weekly <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href...",Good morning #Tokyo #Week42 weekly #airdrop! LAST SO BIG #SpritzCoin $sprtz 1. RT one 2. Fill https://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦ #crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum $BTC $ETH #NewAgepic.twitter.com/ujOUhD8vIy
1,crashfoxcrypto,CrashFoxâ¢,1052710224199671808,2018-10-17 23:57:30,/crashfoxcrypto/status/1052710224199671808,10,2,1,"Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy bitcoin & silver. Vape some kush... It's good karma. Best wishes & regards from ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""4"" lang=""en"">Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy <strong>bitcoin</strong> &amp; silver. Vape some kush... It's good karma. Best wishes &amp; regards from ya boy Crash. <img alt=""â"" aria-label=""Emoji: Victory hand"" class=""Emoji Emoji--forText"" draggable=""false"" src=""https://abs.twimg.com/emoji/v2/72x72/270c.png"" title=""Victory hand""/> <a class=""twitter-tim...","Shit like makes smile. Positivity spreads...do good, good people, stop fucking up, buy bitcoin & silver. Vape kush... It's good karma. Best wishes & regards ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦"
2,AllDef,All Def,1052710193841102849,2018-10-17 23:57:23,/AllDef/status/1052710193841102849,30,12,1,"""Do you take Bitcoin?"" pic.twitter.com/ITv8UdkCZv","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">""Do you take <strong>Bitcoin</strong>?"" <a class=""twitter-timeline-link u-hidden"" data-pre-embedded=""true"" dir=""ltr"" href=""https://t.co/ITv8UdkCZv"">pic.twitter.com/ITv8UdkCZv</a></p>","""Do take Bitcoin?"" pic.twitter.com/ITv8UdkCZv"
3,BitcoinCensus,Dr. Josh Cotton,1052709336357789696,2018-10-17 23:53:58,/BitcoinCensus/status/1052709336357789696,27,2,3,Great to meet you today @MarkYusko during the Digital Asset Strategies Summit! I loved your presentation and your #bullish outlook on #Bitcoin! pic.twitter.com/XcO5Y8fpH4,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Great to meet you today <a class=""twitter-atreply pretty-link js-nav"" data-mentioned-user-id=""254280603"" dir=""ltr"" href=""/MarkYusko""><s>@</s><b>MarkYusko</b></a> during the Digital Asset Strategies Summit! I loved your presentation and your <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/bullish?src=hash""><s>#</s><b>bullish</b></a> outlook on <a class=...",Great meet today @MarkYusko Digital Ass

### Handle Mentions, Urls, Hashtags, Non-Ascii Characters

In [25]:
def preprocess(row):
#   mention
    row = re.sub(r'@[A-Za-z0-9]+',' ',row)
#   url
    row = re.sub('https?://[A-Za-z0-9./]+',' ',row)
#   hashtags
    row = re.sub("[^a-zA-Z]", " ", row)
    return row
#   non-ascii characters
    row = re.sub(r'[^\x00-\x7F]+','',row)
#   \n
    row = re.sub('\n', ' ', row)  
#   \r
    row = re.sub('\r', ' ', row)  
#   /n
    row = re.sub('/n', ' ', row)  
#   /r
    row = re.sub('/r', ' ', row)  
#   replace multiple spaces by one
    row = re.sub(' +', ' ', row)
 

df['step1'] = df['step1'].apply(preprocess)

In [28]:
df

,user,fullname,tweet-id,timestamp,url,likes,replies,retweets,text,html,step1
0,SpritzCoin,SpritzCoin,1052710680200134657,2018-10-17 23:59:19,/SpritzCoin/status/1052710680200134657,13,12,12,Good morning #Tokyo\r\n#Week42 weekly #airdrop!\r\nLAST SO BIG #SpritzCoin $sprtz\r\n1. RT this one\r\n2. Fill in\r\nhttps://docs.google.com/forms/d/e/1FAIpQLSe3niHymrYBg4E_Q6Q05N3DJrp4-qjBhlVlIsVUWdbFzcF8yA/viewform?usp=sf_linkÂ â¦\r\n#crypto #blockchain #bounty #airdropcampaign #investment #airdrops #airdropalert #free #airdropwaves #market #bitcoin #ethereum\r\n$BTC $ETH\r\n#NewAgepic.twitter.com/ujOUhD8vIy,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Good morning <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Tokyo?src=hash""><s>#</s><b>Tokyo</b></a>\r\n<a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/Week42?src=hash""><s>#</s><b>Week42</b></a> weekly <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href...",Good morning Tokyo Week weekly airdrop LAST SO BIG SpritzCoin sprtz RT one Fill Q Q N DJrp qjBhlVlIsVUWdbFzcF yA viewform usp sf link crypto blockchain bounty airdropcampaign investment airdrops airdropalert free airdropwaves market bitcoin ethereum BTC ETH NewAgepic twitter com ujOUhD vIy
1,crashfoxcrypto,CrashFoxâ¢,1052710224199671808,2018-10-17 23:57:30,/crashfoxcrypto/status/1052710224199671808,10,2,1,"Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy bitcoin & silver. Vape some kush... It's good karma. Best wishes & regards from ya boy Crash. https://twitter.com/Sawcruhteez/status/1052701439766626304Â â¦","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""4"" lang=""en"">Shit like this makes me smile. Positivity spreads...do good, be good to people, stop fucking up, buy <strong>bitcoin</strong> &amp; silver. Vape some kush... It's good karma. Best wishes &amp; regards from ya boy Crash. <img alt=""â"" aria-label=""Emoji: Victory hand"" class=""Emoji Emoji--forText"" draggable=""false"" src=""https://abs.twimg.com/emoji/v2/72x72/270c.png"" title=""Victory hand""/> <a class=""twitter-tim...",Shit like makes smile Positivity spreads do good good people stop fucking up buy bitcoin silver Vape kush It s good karma Best wishes regards ya boy Crash
2,AllDef,All Def,1052710193841102849,2018-10-17 23:57:23,/AllDef/status/1052710193841102849,30,12,1,"""Do you take Bitcoin?"" pic.twitter.com/ITv8UdkCZv","<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">""Do you take <strong>Bitcoin</strong>?"" <a class=""twitter-timeline-link u-hidden"" data-pre-embedded=""true"" dir=""ltr"" href=""https://t.co/ITv8UdkCZv"">pic.twitter.com/ITv8UdkCZv</a></p>",Do take Bitcoin pic twitter com ITv UdkCZv
3,BitcoinCensus,Dr. Josh Cotton,1052709336357789696,2018-10-17 23:53:58,/BitcoinCensus/status/1052709336357789696,27,2,3,Great to meet you today @MarkYusko during the Digital Asset Strategies Summit! I loved your presentation and your #bullish outlook on #Bitcoin! pic.twitter.com/XcO5Y8fpH4,"<p class=""TweetTextSize js-tweet-text tweet-text"" data-aria-label-part=""0"" lang=""en"">Great to meet you today <a class=""twitter-atreply pretty-link js-nav"" data-mentioned-user-id=""254280603"" dir=""ltr"" href=""/MarkYusko""><s>@</s><b>MarkYusko</b></a> during the Digital Asset Strategies Summit! I loved your presentation and your <a class=""twitter-hashtag pretty-link js-nav"" data-query-source=""hashtag_click"" dir=""ltr"" href=""/hashtag/bullish?src=hash""><s>#</s><b>bullish</b></a> outlook on <a class=...",Great meet today Digital Asset Strategies Summit I loved presentation bullish outlook Bitcoin pic twitter com XcO Y fpH
4,thecryptokidd,CryptoKid,1052708492899946496,2018-10-17 23:50:37,/thecryptokidd/status/105270849289

### Stemming 

In [ ]:
# ps = PorterStemmer()
# df['stem'] = df['step1'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split()])) 
# print(df['stem'])

### Lemmetization

In [ ]:
# import nltk
# nltk.download('wordnet')
# lmtzr = WordNetLemmatizer()
# df['lem'] = df['step1'].apply(lambda x: ' '.join([lmtzr.lemmatize(word, 'v') for word in x.split()])) 
# print(df['lem'])

### Captilization

In [ ]:
# df['cap'] = df['step1'].apply(lambda x: ' '.join([word.upper() for word in x.split()])) 
# print(df['cap'])

### POS

In [ ]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
 
# df['pos'] = df['step1'].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x))) 
# print(df['pos'])

In [27]:
df.to_csv("..\Data_json_csv\P_12_Oct_24_Oct.csv", index=False)

In [ ]:
# df.head()